<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/keras/cnn/efficientnet_tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**test gpu**

In [ ]:
!nvidia-smi

**install latest tensorflow**

In [ ]:
!pip3 install tf-nightly

**imports**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
import keras

# models
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

# backend
from tensorflow.keras import optimizers, metrics, models
import tensorflow.keras.backend as K

# layers
from tensorflow.keras.layers import Input, add, Conv2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization

# optimizers
from tensorflow.keras.optimizers import SGD, Adam, Adagrad, RMSprop

# regularizers
from tensorflow.keras.regularizers import l2

# training
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# other libs
import math
import pickle
import os

**check keras aplications**

In [ ]:
import tensorflow as tf
dir(tf.keras.applications)

**import keras aplications**

In [ ]:
# keras aplications
from tensorflow.keras.applications import (DenseNet121,
 DenseNet169,
 DenseNet201,
 EfficientNetB0,
 EfficientNetB1,
 EfficientNetB2,
 EfficientNetB3,
 EfficientNetB4,
 EfficientNetB5,
 EfficientNetB6,
 EfficientNetB7,
 InceptionResNetV2,
 InceptionV3,
 MobileNet,
 MobileNetV2,
 NASNetLarge,
 NASNetMobile,
 ResNet101,
 ResNet101V2,
 ResNet152,
 ResNet152V2,
 ResNet50,
 ResNet50V2,
 VGG16,
 VGG19)

**connect to the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**set options**

In [ ]:
image_size = 224 
batch_size = 8
nr_classes = 50
nr_ephocs = 30
train_dir = "/content/drive/My Drive/Datasets/food_data_50/50/train"
test_dir = "/content/drive/My Drive/Datasets/food_data_50/50/test"
all_dir = "/content/drive/My Drive/Datasets/food_data_50/50/all"
model_path = "/content/drive/My Drive/models/efficientnetb7_tfkeras_50_class_model.hdf5"

**generate data**

In [ ]:
train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.3,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )

valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

train = train_datagen.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory(test_dir, target_size=(image_size,image_size), batch_size=batch_size)

# train = train_datagen.flow_from_directory(
#     directory = all_dir,
#     target_size = (image_size,image_size), 
#     batch_size = batch_size,
#     subset="training",
#     shuffle=True
# )

# valid = train_datagen.flow_from_directory(
#     directory = all_dir,
#     target_size = (image_size,image_size), 
#     batch_size = batch_size,
#     subset="validation",
#     shuffle=True
# )

**build model**

In [ ]:
base_model = EfficientNetB0(include_top=False)

In [ ]:
base_model = EfficientNetB7(include_top=False)

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
preds=Dense(nr_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
efficient_net = EfficientNetB7(
    weights='imagenet',
    input_shape=(image_size,image_size,3),
    include_top=False,
    pooling='max'
)

model = Sequential()
model.add(efficient_net)
model.add(Dense(units = 1000, activation='relu'))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(nr_classes,activation='softmax'))
model.summary()

**set optimizer**

In [ ]:
opt = SGD(lr=0.0001, momentum=0.9)

In [ ]:
opt = SGD(lr=0.01, nesterov=True)

In [ ]:
opt = Adam(lr=0.0001)

In [ ]:
opt = Adagrad()

In [ ]:
opt = RMSprop()

**compile model**

In [ ]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'

top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'

model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

**set checkpointer**

In [ ]:
checkpointer = ModelCheckpoint(filepath= model_path, monitor='val_loss', save_best_only=True)

**train**

In [ ]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid, callbacks=[checkpointer])

In [ ]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid)